### Combining GDP data with the jobs report

Brian Dew

Updated: September 20, 2020

----

Notes:

The basic idea here is to use the BEA population estimates plus CPS employment rate and hours worked trends to estimate GDP per hour of work. BLS does this process for its productivity and costs report, using much more comprehensive data, however this approximation has proven decent over time.

The hard part to this measure is getting hours worked right. All of the published measures are either too-low-frequency or don't have a broad enough definition of workers. I want to capture all workers, regardless of full-time or part-time status or of whether they work for the private sector. Also want to capture second and third jobs. As a result, I selected total actual hours worked from the CPS microdata, specifically finding the trend using x13as with default settings. Hours worked from the CPS microdata have issues around holidays falling in the reference week, and also do not capture hours worked for some important categories of labor such as self-employed persons.

In [1]:
%matplotlib inline
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrs = {1: 1, 2: 1, 3: 1, 4: 2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}

In [2]:
aah = {}
epop = {}
cols = ['HRSACTT', 'LFS', 'MONTH']
for year in range(1989, 2022):
    if year >= 1998:
        wgt = 'PWSSWGT'
    else:
        wgt = 'BASICWGT'
    df = pd.read_feather(cps_dir / f'cps{year}.ft', columns=cols + [wgt]).rename({wgt: 'WGT'}, axis=1)
    ah = (df.query('LFS == "Employed"')
            .groupby('MONTH').apply(lambda x: np.average(x.HRSACTT.replace(-1, 0), weights=x.WGT)))
    aah.update({pd.to_datetime(f'{year}-{month}-01'): value for month, value in list(zip(ah.index, ah.values))})
    
    ep = (df.query('LFS == "Employed"').groupby('MONTH').WGT.sum() / 
          df.groupby('MONTH').WGT.sum())
    epop.update({pd.to_datetime(f'{year}-{month}-01'): value for month, value in list(zip(ep.index, ep.values))})

In [3]:
sm = x13_arima_analysis(pd.Series(epop))
epop_sa = sm.seasadj.resample('QS').mean()
sm = x13_arima_analysis(pd.Series(aah))
aah_sa = sm.trend.resample('QS').mean()

/home/brian/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/x13.py:187: X13Warning: NOTE: Unable to test AO2020.Apr due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test AO2020.May due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test LS2020.Mar due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test AO2020.Apr due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test LS2020.May d

In [4]:
gdp_code = ('T10106', ['A191RX'])
pop_code = ('T70100', ['B230RC'])
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
gdp = rgdp / rgdp.iloc[-1] * cd
pop = nipa_df(retrieve_table(pop_code[0])['Data'], pop_code[1]).sort_index()

df = pd.DataFrame()
df['epop'] = epop_sa * 100
df['pop'] = pop['B230RC'] / 1000
#df['hours'] = fred_df('PRS84006023')
df['hours'] = aah_sa
df['gdp'] = gdp['A191RX']
df['input'] = (df['pop'] * (df['epop'] / 100)) * (df['hours'] * 52)
df['gdpinp'] = df['gdp'] / df['input']
datetxt = dtxt(df['gdpinp'].dropna().index[-1])['qtr1']
write_txt(text_dir / 'gdpjobs_ltdt.txt', datetxt)
df.to_csv(data_dir / 'gdpjobslvl.csv', index_label='date')

df['epop_c'] = ((df['pop'] * df['epop'].mean()) * df['hours']) * df['gdpinp']
df['pop_c'] = ((df['pop'].mean() * df['epop']) * df['hours']) * df['gdpinp']
df['hours_c'] = ((df['pop'] * df['epop']) * df['hours'].mean()) * df['gdpinp']

df_g = growth_rate(df.dropna())

df_g['pop_contr'] = df_g['gdp'] - df_g['pop_c']
df_g['epop_contr'] = df_g['gdp'] - df_g['epop_c']
df_g['hours_contr'] = df_g['gdp'] - df_g['hours_c']
df_g['prod'] = df_g['gdp'] - df_g['pop_contr'] - df_g['epop_contr'] - df_g['hours_contr']

result = df_g[['pop_contr', 'epop_contr', 'hours_contr', 'prod']].round(2)

result.to_csv(data_dir / 'gdpjobs.csv', index_label='date')

In [5]:
data = df.dropna()

ltval = f"\${data['gdpinp'].iloc[-1]:.2f}"
prval = f"\${data['gdpinp'].iloc[-2]:.2f}"
prdate = dtxt(data['gdpinp'].index[-2])['qtr1']
prdt = '2019-10-01'
prval2 = f"\${data.loc[prdt, 'gdpinp']:.2f}"
prval3 = f"\${data.loc['2015-10-01', 'gdpinp']:.2f}"
val89 = f"\${data.loc['1989-01-01', 'gdpinp']:.2f}"
gdpval = f"\${data['gdp'].iloc[-1] / 1_000:,.0f}"
gdppr = f"\${data.loc[prdt, 'gdp'] / 1_000:,.0f}"
agghrs = f"{data['input'].iloc[-1] / 1_000:,.0f}"
hrspr = f"{data.loc[prdt, 'input'] / 1_000:,.0f}"

text = (f'In {datetxt}, real GDP was equivalent to roughly {ltval} per hour of '+
        f'work, compared to {prval} in {prdate}, {prval2} in 2019 Q4, {prval3} '+
        f'in 2015 Q4, and {val89} in the first quarter of 1989. Comparing the latest '+
        'data to the pre-COVID data covering 2019 Q4, annualized real GDP '+
        f'is {gdpval} billion in the latest data and '+
        f'{gdppr} billion in 2019 Q4. Aggregate hours worked total '+
        f'{agghrs} billion in the latest quarter and {hrspr} billion '+
        'in 2019 Q4.')
write_txt(text_dir / 'gdp_per_hour.txt', text)
print(text, '\n\n')

poplt = value_text(result['pop_contr'].iloc[-1], style='contribution_to', ptype='pp', digits=2)
poppr = value_text(result.loc[prdt, 'pop_contr'], style='contribution', ptype='pp', digits=2, casual=True)
emplt = value_text(result['epop_contr'].iloc[-1], style='contribution', ptype='pp', digits=2)
emppr = value_text(result.loc[prdt, 'epop_contr'], style='contribution', ptype='pp', digits=2, casual=True)
hrslt = value_text(result['hours_contr'].iloc[-1], style='contribution_to', ptype='pp', digits=2, casual=True)
hrspr = value_text(result.loc[prdt, 'hours_contr'], style='contribution', ptype='pp', digits=2, casual=True)
prodlt = value_text(result['prod'].iloc[-1], style='contribution', ptype='pp', digits=2)
prodpr = value_text(result.loc[prdt, 'prod'], style='contribution_of', ptype='pp', digits=2, casual=True)

text = (f'In {datetxt}, population growth {poplt} annualized GDP growth, and, for comparison, '+
        f'{poppr} in 2019 Q4. '+
        f'Changes in the employed share of the population {emplt} in the latest quarter, and '+
        f'{emppr} in the fourth quarter of 2019. Changes in average hours worked {hrslt} GDP '+
        f'growth in the latest quarter and {hrspr} in 2019 Q4. Lastly, productivity {prodlt} '+
        f'to GDP growth in {datetxt}, compared to {prodpr} in 2019 Q4.')
write_txt(text_dir / 'gdpjobsch.txt', text)
print(text)

In 2021 Q3, real GDP was equivalent to roughly \$76.64 per hour of work, compared to \$76.84 in 2021 Q2, \$72.20 in 2019 Q4, \$69.94 in 2015 Q4, and \$46.66 in the first quarter of 1989. Comparing the latest data to the pre-COVID data covering 2019 Q4, annualized real GDP is \$23,173 billion in the latest data and \$22,861 billion in 2019 Q4. Aggregate hours worked total 302 billion in the latest quarter and 317 billion in 2019 Q4. 


In 2021 Q3, population growth contributed 0.37 percentage point to annualized GDP growth, and, for comparison, added 0.56 percentage point in 2019 Q4. Changes in the employed share of the population contributed 2.61 percentage points in the latest quarter, and added 1.64 percentage points in the fourth quarter of 2019. Changes in average hours worked added 0.08 percentage point to GDP growth in the latest quarter and added 0.36 percentage point in 2019 Q4. Lastly, productivity subtracted 1.05 percentage points to GDP growth in 2021 Q3, compared to a reduc

In [6]:
df

,epop,pop,hours,gdp,input,gdpinp,epop_c,pop_c,hours_c
1989-01-01,48.433067,246.460,37.432008,1.084234e+07,232345.833232,46.664672,2.062015e+07,2.484706e+07,2.084113e+07
1989-04-01,48.397997,247.017,37.458785,1.092510e+07,232868.772368,46.915256,2.079259e+07,2.498025e+07,2.098519e+07
1989-07-01,48.459407,247.698,37.481855,1.100604e+07,233951.055598,47.044197,2.092010e+07,2.509614e+07,2.112766e+07
1989-10-01,48.424753,248.374,37.485228,1.102773e+07,234442.877342,47.038013,2.097632e+07,2.507715e+07,2.116738e+07
1990-01-01,48.347388,248.936,37.479783,1.114824e+07,234563.878749,47.527512,2.123948e+07,2.529396e+07,2.140181e+07
...,...,...,...,...,...,...,...,...,...
2020-07-01,45.112721,330.368,36.842513,2.209677e+07,285528.578325,77.389011,4.511703e+07,3.777715e+07,4.315399e+07
2020-10-01,46.298417,330.815,37.002814,2.234321e+07,294706.304551,75.815190,4.445189e+07,3.814686e+07,4.344624e+07
2021-01-01,46.482143,331.011,37.153992,2.268593e+07,297260.629390,76.316648,4.495533e+07,3.870906e+07,4.393317e+07
2021-04-01,46.805721,331.209,37.226676,2.305815e+07,300094.938123,76.836196,4.537705e+07,3.932065e+07,4.456681e+07
